In [1]:
from __future__ import division
from IPython.display import display

import time
t = time.time    

#import sympy as sym
#from sympy import *

import symengine
from symengine import *
from sympy import simplify, transpose
from sympy import Matrix
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


g = symbols('g')
a1, a2, a3, a4, a5 = symbols("a_1 a_2 a_3 a_4 a_5")
d1, d2, d3, d4, d5 = symbols("d_1 d_2 d_3 d_4 d_5")
q = symbols("q_1 q_2 q_3 q_4 q_5")
dq = symbols('\dot{q_1} \dot{q_2} \dot{q_3} \dot{q_4} \dot{q_5}')
ddq = symbols('\ddot{q_1} \ddot{q_2} \ddot{q_3} \ddot{q_4} \ddot{q_5}')
m = symbols('m_1 m_2 m_3 m_4 m_5')
Ixx = symbols('I_{1_{xx}} I_{2_{xx}} I_{3_{xx}} I_{4_{xx}} I_{5_{xx}}')
Iyy = symbols('I_{1_{yy}} I_{2_{yy}} I_{3_{yy}} I_{4_{yy}} I_{5_{yy}}')
Izz = symbols('I_{1_{zz}} I_{2_{zz}} I_{3_{zz}} I_{4_{zz}} I_{5_{zz}}')
Ixy = symbols('I_{1_{xy}} I_{2_{xy}} I_{3_{xy}} I_{4_{xy}} I_{5_{xy}}')
Ixz = symbols('I_{1_{xz}} I_{2_{xz}} I_{3_{xz}} I_{4_{xz}} I_{5_{xz}}')
Iyz = symbols('I_{1_{yz}} I_{2_{yz}} I_{3_{yz}} I_{4_{yz}} I_{5_{yz}}')

xc = symbols('x_{c0} x_{c1} x_{c2} x_{c3} x_{c4} x_{c5}')
yc = symbols('y_{c0} y_{c1} y_{c2} y_{c3} y_{c4} y_{c5}')
zc = symbols('z_{c0} z_{c1} z_{c2} z_{c3} z_{c4} z_{c5}')
x = symbols('x_{0} x_{1} x_{2} x_{3} x_{4} x_{5}')
y = symbols('y_{0} y_{1} y_{2} y_{3} y_{4} y_{5}')
z = symbols('z_{0} z_{1} z_{2} z_{3} z_{4} z_{5}')

ai = [a1, a2, a3, 0, 0]
di = [d1, 0, 0, 0, d5]
alphai = [pi/2, 0, 0, pi/2, 0]
thi = [q[0],q[1],q[2],q[3],q[4]]

n = 5
g0 = Matrix([[0],[0],[-g]])

"init tensors"
tensors = [-1 for j in range(n+1)]
for i in range(0, n):
    tensors[i] = Matrix([[Ixx[i], Ixy[i], Ixz[i]],
                         [Ixy[i], Iyy[i], Iyz[i]],
                         [Ixz[i], Iyz[i], Izz[i]]])

"initialization of arrays for acceleration of computations"
Ts = [[-1 for j in range(n+1)] for i in range(n+1)]
Rs = [[-1 for j in range(n+1)] for i in range(n+1)]
Zs = [-1 for j in range(n+1)]
"for accelerations of arm"
OMEGAs = [-1 for j in range(n + 1)]
DOMEGAs = [-1 for j in range(n + 1)]
As = [-1 for j in range(n + 1)]
ACs = [-1 for j in range(n + 1)]
"for forces/torques of arm"
Gs = [-1 for j in range(n + 2)]
Fs = [-1 for j in range(n + 2)]
TQs = [-1 for j in range(n + 2)]
FP1s = [-1 for j in range(n + 2)]
TQP1s = [-1 for j in range(n + 2)]


# dont remember where need add theta_2 (defaut)
def T(i, j, simp=True):
    if Ts[i][j] != -1:
        return Ts[i][j]
    else:
        H = eye(4)
        for k in range(i, j):
            Rz = Matrix([[cos(thi[i]), -sin(thi[i]), 0, 0],[sin(thi[i]), cos(thi[i]), 0, 0],[0,0,1,0],[0,0,0,1]])
            Rx = Matrix([[1, 0, 0, 0],[0, cos(alphai[i]), -sin(alphai[i]), 0],[0,sin(alphai[i]), cos(alphai[i]), 0],[0,0,0,1]])
            Tz = Matrix([[1,0,0,0],[0,1,0,0],[0,0,1, di[i]],[0,0,0,1]])
            Tx = Matrix([[1,0,0,ai[i]],[0,1,0,0],[0,0,1, 0],[0,0,0,1]])
            dT = Rz*Tz*Tx*Rx
            H = H * dT
        if simp:
            H = simplify(expand(H))
        Ts[i][j] = H
        return H

def R(i, j, simp=True):
    if Rs[i][j] != -1:
        return Rs[i][j]
    else:
        r = T(i,j)[:3, :3]
        if simp:
            r = simplify(expand(r))
        Rs[i][j] = r
        return r

def getZ(i, simp=True):
    "in 0 RF"
    if Zs[i] != -1:
        return Zs[i]
    else:
        z = R(0, i) * Matrix([[0],[0],[1]])
        Zs[i] = z
        return z

def getOmega(i, simp=True):
    if OMEGAs[i] != -1:
        return OMEGAs[i]
    else:
        omega = 0
        if i > 0:
            omega = transpose(R(i-1, i)) * (getOmega(i-1, simp) + dq[i-1] * getZ(i-1))
        else:
            omega = Matrix([[0],[0],[0]])
        if simp:
            omega = simplify(expand(omega))
        OMEGAs[i] = omega
        return omega

def getDOmega(i, simp=True):
    if DOMEGAs[i] != -1:
        return DOMEGAs[i]
    else:
        dOmega = 0
        if i > 0:
            dOmega = transpose(R(i-1, i)) * (getDOmega(i-1, simp) + ddq[i-1] * getZ(i-1) + (dq[i-1] * getOmega(i-1), simp).cross(getZ(i-1)))
        else:
            dOmega = Matrix([[0],[0],[0]])
        if simp:
            dOmega = simplify(expand(dOmega))
        DOMEGAs[i] = dOmega
        return dOmega

# r vectros    
def getri(i, simp=True):
    "r in i RF"
    ri = transpose(R(i-1, i)) * T(i-1, i)[:3, 3]
    if simp:
        ri = simplify(expand(ri))
    return ri # 3x1

def getrci(i):
    "rc in i RF"
    return Matrix([[xc[i]],[yc[i]],[zc[i]], [1]]) # 4x1
    
    
def getrc0(i, simp=True):
    "rc in 0 RF"
    rc0 = T(0, i) * getrci(i)
    if simp:
        rc0 = simplify(expand(rc0))
    return rc0 # 3x1

def geta(i, simp=True):
    if As[i] != -1:
        return As[i]
    else:
        a = 0
        if i > 0:
            omega = getOmega(i, simp)
            dOmega = getDOmega(i, simp)
            ri = getri(i)
            a = transpose(R(i-1, i)) * geta(i-1, simp) + dOmega.cross(ri) + omega.cross(omega.cross(ri)) 
        else:
            a = Matrix([[0],[0],[0]])
        if simp:
            a = simplify(expand(a))
        As[i] = a
        return a

def getac(i, simp=True):
    if ACs[i] != -1:
        return ACs[i]
    else:
        ac = 0
        if i > 0:
            omega = getOmega(i, simp)
            dOmega = getDOmega(i, simp)
            rci = getrci(i)[:3,:3] # colomn 3x1
            ac = geta(i, False) + dOmega.cross(rci) + omega.cross(omega.cross(rci)) 
        else:
            ac = Matrix([[0],[0],[0]])
        if simp:
            ac = simplify(expand(ac))
        ACs[i] = ac
        return ac
    
    
def getG(i, simp=True):
    if Gs[i] != -1:
        return Gs[i]
    else:
        g = 0
        if i < n:
            g = transpose(R(i, i+1)) * getG(i+1, simp)
        else:
            g = transpose(R(0, n)) * g0
        if simp:
            g = simplify(expand(g))
        Gs[i] = g
        return g
    
def getTensorC(i):
    return tensors[i-1]
        
def getFp1(i, simp=True):
    " f from i to i+1 (p1 = plus 1)"
    if FP1s[i] != -1:
        return FP1s[i]
    else:
        f = 0
        if i < n:
            f = R(i, i+1) * getF(i+1, simp) 
        else:
            f = Matrix([[0],[0],[0]])
        if simp:
            f = simplify(expand(f))
        FP1s[i] = f
        return f
    
def getF(i, simp=True):
    startTime = t()
    print(str(startTime) + ': getting F ' + str(i))
    if Fs[i] != -1:
        endTime = t()
        print(str(endTime) + ': got F ' + str(i) + ':from store!')
        return Fs[i]
    else:
        f = 0
        if i <= n:
            f = getFp1(i, simp) + m[i-1] * (getac(i, simp) - getG(i, simp))
        if simp:
            f = simplify(expand(f))
        Fs[i] = f
        endTime = t()
        print(str(endTime) + ': got F ' + str(i) + ':computed!')
        return f

def getTqp1(i, simp=True):
    print('getting Tqp1 ' + str(i))
    " torque from i to i+1 (p1 = plus 1)"
    if TQP1s[i] != -1:
        print('getting Tqp1 ' + str(i) + ':from store!')
        return TQP1s[i]
    else:
        tq = 0
        if i < n:
            tq = R(i, i+1) * getTq(i+1) 
        else:
            tq = Matrix([[0],[0],[0]])
        if simp:
            tq = simplify(expand(tq))
        TQP1s[i] = tq
        print('getting Tqp1 ' + str(i) + ':computed!')
        return tq

def getTq(i, simp=True):
    startTime = t()
    print(str(startTime) + ': getting TQ ' + str(i))
    if TQs[i] != -1:
        endTime = t()
        print(str(endTime) + ': got TQ ' + str(i) + ': from store!')
        return TQs[i]
    else:
        tq = 0
        if i <= n:
            ri = getri(i)
            rci = getrci(i)[:3,:3] # colomn 3x1
            tq = getTqp1(i, simp) - getF(i, simp).cross(ri - rci) + getFp1(i, simp).cross(ri) + getTensorC(i) * getDOmega(i) + getOmega(i, simp).cross(getTensorC(i) * getOmega(i, simp))
        if simp:
            tq = simplify(expand(tq))
        TQs[i] = tq
        endTime = t()
        print(str(endTime) + ': got TQ ' + str(i) + ': computed!')
        return tq


def directRecursion(simp=False):
    startTime = t()
    for i in range(n+1):
        o = getOmega(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('Omega_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    startTime = t()
    for i in range(n+1):
        o = getDOmega(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('DOmega_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    startTime = t()
    for i in range(n+1):
        o = geta(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('a_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    startTime = t()
    for i in range(n+1):
        o = getac(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('ac_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    

def backwardRecurion(simp=False):
    startTime = t()
    for i in range(n, 0, -1):
        o = getF(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('F_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    startTime = t()
    for i in range(n, 0, -1):
        o = getTq(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('Tq_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')

# %%!%store -r

In [ ]:
#directRecursion()

In [ ]:
%store -r

In [ ]:
temp = getF(0, False)

In [16]:
def backwardRecurion(simp=False):
    startTime = t()
    for i in range(n, 0, -1):
        o = getF(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('F_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')
    startTime = t()
    for i in range(n, 0, -1):
        o = getTq(i, simp)
        endTime = t()
        totalTime = round(endTime - startTime, 3)
        minutes = totalTime // 60
        seconds = totalTime % 60
        print('Tq_' + str(i) + ' was computed for ' + str(minutes) + ' min. ' + str(seconds) + ' sec.')

In [ ]:
tq5 = simplify(TQs[5])

In [4]:
import symengine
from symengine import *
q = symbols('q')

In [23]:
from symengine import simp
from sympy import symplify

ImportError: cannot import name 'symplify'

In [38]:
%store -r

In [40]:
tq5 = expand(TQs[5])

In [ ]:
tq5 = simplify(tq5)